In [35]:
import requests
import json
from datetime import datetime
from dotenv import load_dotenv
import os
import time
import pandas as pd

load_dotenv()  # Carrega .env

OPENWEATHER_API_KEY = os.getenv('OPENWEATHER_API_KEY')
OPENAQ_API_KEY = os.getenv('OPENAQ_API_KEY')

CITIES = ['São Paulo', 'Rio de Janeiro', 'Rondonópolis']

In [36]:
def fetch_openweather(city):
    print(f"Buscando dados do OpenWeather para {city}")
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={OPENWEATHER_API_KEY}"
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fetch OpenWeather para {city}: {e}")
        return None        
                    
teste = fetch_openweather ('São Paulo')

print(teste)

Buscando dados do OpenWeather para São Paulo
{'coord': {'lon': -46.6361, 'lat': -23.5475}, 'weather': [{'id': 800, 'main': 'Clear', 'description': 'clear sky', 'icon': '01d'}], 'base': 'stations', 'main': {'temp': 300.33, 'feels_like': 300.31, 'temp_min': 299.9, 'temp_max': 302.09, 'pressure': 1014, 'humidity': 43, 'sea_level': 1014, 'grnd_level': 926}, 'visibility': 10000, 'wind': {'speed': 6.69, 'deg': 360}, 'clouds': {'all': 0}, 'dt': 1755704467, 'sys': {'type': 1, 'id': 8394, 'country': 'BR', 'sunrise': 1755682094, 'sunset': 1755723114}, 'timezone': -10800, 'id': 3448439, 'name': 'São Paulo', 'cod': 200}


In [37]:
#Buscar dados openaq

def fetch_openaq_locations():
    print("Buscando todos os locais do OpenAQ para o Brasil")
    url = "https://api.openaq.org/v3/locations?countries_id=45"
    headers = {
        'X-API-Key': OPENAQ_API_KEY,
        'Content-Type': 'application/json'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fetch OpenAQ locations: {e}")
        return None
    
    
locations = fetch_openaq_locations()

print(locations)
    

Buscando todos os locais do OpenAQ para o Brasil
{'meta': {'name': 'openaq-api', 'website': '/', 'page': 1, 'limit': 100, 'found': 87}, 'results': [{'id': 5012, 'name': 'Diadema', 'locality': 'Diadema', 'timezone': 'America/Sao_Paulo', 'country': {'id': 45, 'code': 'BR', 'name': 'Brazil'}, 'owner': {'id': 4, 'name': 'Unknown Governmental Organization'}, 'provider': {'id': 220, 'name': 'Sao Paulo CETESB'}, 'isMobile': False, 'isMonitor': True, 'instruments': [{'id': 2, 'name': 'Government Monitor'}], 'sensors': [{'id': 12743, 'name': 'o3 µg/m³', 'parameter': {'id': 3, 'name': 'o3', 'units': 'µg/m³', 'displayName': 'O₃ mass'}}, {'id': 12744, 'name': 'pm10 µg/m³', 'parameter': {'id': 1, 'name': 'pm10', 'units': 'µg/m³', 'displayName': 'PM10'}}], 'coordinates': {'latitude': -23.68587641, 'longitude': -46.61162193}, 'licenses': None, 'bounds': [-46.61162193, -23.68587641, -46.61162193, -23.68587641], 'distance': None, 'datetimeFirst': {'utc': '2017-07-01T00:00:00Z', 'local': '2017-06-30T21:

In [38]:
def fetch_openaq_sensors(location_id):
    print(f"Buscando sensores para o local {location_id}")
    url = f"https://api.openaq.org/v3/locations/{location_id}/sensors"
    headers = {
        'X-API-Key': OPENAQ_API_KEY,
        'Content-Type': 'application/json'
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Erro ao fetch sensores do OpenAQ para o local {location_id}: {e}")
        return None
    
    
data = {'openweather': {}, 'openaq': {}}

openaq_locations = fetch_openaq_locations()

all_sensor_data = []
if openaq_locations and 'results' in openaq_locations:
    for location in openaq_locations['results']:
        if location.get('locality') in CITIES:
            time.sleep(1) # to avoid hitting rate limits
            sensor_data = fetch_openaq_sensors(location['id'])
            if sensor_data and 'results' in sensor_data:
                for sensor in sensor_data['results']:
                    # Add location info to each sensor reading
                    sensor['locality'] = location.get('locality')
                    sensor['location_name'] = location.get('name')
                    all_sensor_data.append(sensor)

print(all_sensor_data)


Buscando todos os locais do OpenAQ para o Brasil
Buscando sensores para o local 5233
Buscando sensores para o local 5243
Buscando sensores para o local 5252
Buscando sensores para o local 5258
Buscando sensores para o local 5268
Buscando sensores para o local 5277
Buscando sensores para o local 5278
Buscando sensores para o local 5279
Buscando sensores para o local 5282
Buscando sensores para o local 5284
Buscando sensores para o local 5288
Buscando sensores para o local 5289
Buscando sensores para o local 5299
Buscando sensores para o local 5301
Buscando sensores para o local 820321
Buscando sensores para o local 820322
Buscando sensores para o local 820323
Buscando sensores para o local 820324
Buscando sensores para o local 820326
Buscando sensores para o local 820328
Buscando sensores para o local 820329
Buscando sensores para o local 5110472
Buscando sensores para o local 5110473
Buscando sensores para o local 5110474
Buscando sensores para o local 5110511
Buscando sensores para o 

In [39]:

for city in CITIES:
    time.sleep(1)
    data['openweather'][city] = fetch_openweather(city)

data['openaq'] = {city: [s for s in all_sensor_data if s.get('locality') == city] for city in CITIES}

print(data['openweather'][city])

Buscando dados do OpenWeather para São Paulo
Buscando dados do OpenWeather para Rio de Janeiro
Buscando dados do OpenWeather para Rondonópolis
{'coord': {'lon': -54.6356, 'lat': -16.4708}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04d'}], 'base': 'stations', 'main': {'temp': 305.37, 'feels_like': 303.28, 'temp_min': 305.37, 'temp_max': 305.37, 'pressure': 1011, 'humidity': 19, 'sea_level': 1011, 'grnd_level': 977}, 'visibility': 10000, 'wind': {'speed': 1.73, 'deg': 229, 'gust': 1.74}, 'clouds': {'all': 99}, 'dt': 1755704741, 'sys': {'country': 'BR', 'sunrise': 1755683600, 'sunset': 1755725447}, 'timezone': -14400, 'id': 3450909, 'name': 'Rondonópolis', 'cod': 200}


In [40]:
print(data['openaq'])

{'São Paulo': [{'id': 13548, 'name': 'pm25 µg/m³', 'parameter': {'id': 2, 'name': 'pm25', 'units': 'µg/m³', 'displayName': 'PM2.5'}, 'datetimeFirst': {'utc': '2017-07-01T00:00:00Z', 'local': '2017-06-30T21:00:00-03:00'}, 'datetimeLast': {'utc': '2023-04-05T20:00:00Z', 'local': '2023-04-05T17:00:00-03:00'}, 'coverage': {'expectedCount': 1, 'expectedInterval': '01:00:00', 'observedCount': 17883, 'observedInterval': '17883:00:00', 'percentComplete': 1788300.0, 'percentCoverage': 1788300.0, 'datetimeFrom': {'utc': '2017-07-01T00:00:00Z', 'local': '2017-06-30T21:00:00-03:00'}, 'datetimeTo': {'utc': '2023-04-05T20:00:00Z', 'local': '2023-04-05T17:00:00-03:00'}}, 'latest': {'datetime': {'utc': '2023-04-05T20:00:00Z', 'local': '2023-04-05T17:00:00-03:00'}, 'value': 22.0, 'coordinates': {'latitude': -23.56634178, 'longitude': -46.73741428}}, 'summary': {'min': 0.0, 'q02': None, 'q25': None, 'median': None, 'q75': None, 'q98': None, 'max': 127.0, 'avg': 14.011127886819885, 'sd': None}, 'locality

In [41]:
df = pd.DataFrame([{
    'city': city,
    'temp': round(data['main']['temp'] - 273.15, 2) if data and 'main' in data else None,
    'humidity': data['main']['humidity'] if data and 'main' in data else None,
    'weather_desc': data['weather'][0]['description'] if data and data.get('weather') else None,
    'dt': datetime.fromtimestamp(data['dt']) if data and 'dt' in data else None
} 
for city, data in data['openweather'].items()])

df = df.dropna(subset=['temp'])

df['weather_category'] = df['weather_desc'].apply(lambda x: 'Clear' if 'clear' in str(x).lower() else 'Other')
    
df
    

,city,temp,humidity,weather_desc,dt,weather_category
0,São Paulo,27.18,43,clear sky,2025-08-20 11:41:07,Clear
1,Rio de Janeiro,28.26,44,clear sky,2025-08-20 11:45:42,Clear
2,Rondonópolis,32.22,19,overcast clouds,2025-08-20 11:45:41,Other


In [42]:
sensor_readings = []
for city, sensors in data['openaq'].items():
    for sensor in sensors:
        if sensor.get('latest'): # Ensure there is a latest reading
            sensor_readings.append({
                'city': city,
                'location_name': sensor.get('location_name'),
                'parameter': sensor['parameter']['name'],
                'value': sensor['latest']['value'],
            })


df = pd.DataFrame(sensor_readings)

# Pivot the table to have pollutants as columns
df_pivot = df.pivot_table(index=['city', 'location_name'], columns='parameter', values='value').reset_index()

df_pivot

parameter,city,location_name,co,no,no2,nox,o3,pm10,pm25,so2
0,Rio de Janeiro,Bangu,NaN,9.48,27.57,37.05,73.36,41.17,NaN,24.38
1,Rio de Janeiro,Campinho,NaN,NaN,NaN,NaN,NaN,17.38,7.84,NaN
2,Rio de Janeiro,Campo Grande,NaN,11.51,24.80,36.31,139.09,24.83,NaN,2.58
3,Rio de Janeiro,Campo de Santana,NaN,NaN,NaN,NaN,NaN,15.89,6.93,NaN
4,Rio de Janeiro,Cascadura,NaN,NaN,NaN,NaN,NaN,21.78,8.60,NaN
5,Rio de Janeiro,Centro,0.05,NaN,NaN,NaN,82.29,45.17,NaN,NaN
6,Rio de Janeiro,Copacabana,NaN,NaN,NaN,NaN,42.35,38.33,NaN,NaN
7,Rio de Janeiro,Engenheiro Leal,NaN,NaN,NaN,NaN,NaN,20.18,7.03,NaN
8,Rio de Janeiro,Irajá,0.33,12.16,44.20,56.36,51.25,37.00,5.50,1.68
9,Rio de Janeiro,Madureira,NaN,NaN,NaN,NaN,NaN,28.14,9.11,NaN
